In [406]:
#loading dependencies
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # Data visualization
import seaborn as sns # Advanced data visualization

import os
for dirname, _, filenames in os.walk('/kaggle/input/ecommerce-data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/ecommerce-data/data.csv


In [407]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [408]:
#loading data 
ecomdata = pd.read_csv('/kaggle/input/ecommerce-data/data.csv', encoding='ISO-8859-1')


 2. Data preprocessing and EDA

In [409]:
ecomdata.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [410]:
ecomdata.shape

(541909, 8)

In [411]:
ecomdata.describe()

,Quantity,UnitPrice,CustomerID
count,541909.000000,541909.000000,406829.000000
mean,9.552250,4.611114,15287.690570
std,218.081158,96.759853,1713.600303
min,-80995.000000,-11062.060000,12346.000000
25%,1.000000,1.250000,13953.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16791.000000
max,80995.000000,38970.000000,18287.000000


In [412]:
ecomdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [413]:
ecomdata.columns = ecomdata.columns.str.strip()  # Strip any leading/trailing spaces from column names
ecomdata['InvoiceDate'] = pd.to_datetime(ecomdata['InvoiceDate'])

In [414]:
# Drop the StockCode column
ecomdata.drop('StockCode', axis=1, inplace=True)

# Convert the InvoiceNo column to string
ecomdata['InvoiceNo'] = ecomdata['InvoiceNo'].astype('str')


In [415]:
#dropping 
ecomdata.drop_duplicates()
to_drop = ecomdata[ecomdata.InvoiceNo.str.match('^[a-zA-Z]')].index

In [416]:
ecomdata["Country"].value_counts()

Country
United Kingdom          495478
Germany                   9495
France                    8557
EIRE                      8196
Spain                     2533
Netherlands               2371
Belgium                   2069
Switzerland               2002
Portugal                  1519
Australia                 1259
Norway                    1086
Italy                      803
Channel Islands            758
Finland                    695
Cyprus                     622
Sweden                     462
Unspecified                446
Austria                    401
Denmark                    389
Japan                      358
Poland                     341
Israel                     297
USA                        291
Hong Kong                  288
Singapore                  229
Iceland                    182
Canada                     151
Greece                     146
Malta                      127
United Arab Emirates        68
European Community          61
RSA                         58


In [417]:
mybasket = (ecomdata[ecomdata['Country'] == "Germany"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [428]:
mybasket

Description,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,...,YULETIDE IMAGES GIFT WRAP SET,ZINC HEART T-LIGHT HOLDER,ZINC STAR T-LIGHT HOLDER,ZINC BOX SIGN HOME,ZINC FOLKART SLEIGH BELLS,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC WILLIE WINKIE CANDLE STICK
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536527,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536840,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536967,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536983,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C580313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C580714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C580740,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [429]:
# Function to encode positive values as 1, non-positive as 0
def my_encode_units(x):
    return 1 if x > 0 else 0

# Apply the function element-wise to the DataFrame using apply
my_basket_sets = mybasket.apply(lambda x: x.map(my_encode_units))

# Remove the 'POSTAGE' column from the DataFrame
my_basket_sets.drop('POSTAGE', inplace=True, axis=1)

# Display the resulting DataFrame
print(my_basket_sets)



Description   50'S CHRISTMAS GIFT BAG LARGE   DOLLY GIRL BEAKER  \
InvoiceNo                                                         
536527                                    0                   0   
536840                                    0                   0   
536861                                    0                   0   
536967                                    0                   0   
536983                                    0                   0   
...                                     ...                 ...   
C580313                                   0                   0   
C580714                                   0                   0   
C580740                                   0                   0   
C580989                                   0                   0   
C581409                                   0                   0   

Description   I LOVE LONDON MINI BACKPACK   RED SPOT GIFT BAG LARGE  \
InvoiceNo                                                

In [430]:
# Convert the DataFrame to boolean (True for 1, False for 0)
my_basket_sets_bool = my_basket_sets.astype(bool)

# Generate frequent itemsets
my_frequent_itemsets = apriori(my_basket_sets_bool, min_support=0.05, use_colnames=True)

# Display the frequent itemsets
print(my_frequent_itemsets)


     support                                           itemsets
0   0.077944                           (6 RIBBONS RUSTIC CHARM)
1   0.053068                        (ALARM CLOCK BAKELIKE PINK)
2   0.054726                                (GUMBALL COAT RACK)
3   0.069652                           (JAM MAKING SET PRINTED)
4   0.059701                          (JUMBO BAG RED RETROSPOT)
5   0.076285                       (JUMBO BAG WOODLAND ANIMALS)
6   0.059701                               (LUNCH BAG WOODLAND)
7   0.064677                  (PACK OF 72 RETROSPOT CAKE CASES)
8   0.087894                   (PLASTERS IN TIN CIRCUS PARADE )
9   0.081260                         (PLASTERS IN TIN SPACEBOY)
10  0.053068                        (PLASTERS IN TIN STRONGMAN)
11  0.104478                 (PLASTERS IN TIN WOODLAND ANIMALS)
12  0.053068                      (RED RETROSPOT CHARLOTTE BAG)
13  0.053068                                (RED RETROSPOT CUP)
14  0.072968                    (RED TOA

In [431]:
#generating rules
my_rules = association_rules(my_frequent_itemsets, metric="lift", min_threshold=1)

In [432]:
my_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN CIRCUS PARADE ),0.104478,0.087894,0.051410,0.492063,5.598383,0.042227,1.795709,0.917204
1,(PLASTERS IN TIN CIRCUS PARADE ),(PLASTERS IN TIN WOODLAND ANIMALS),0.087894,0.104478,0.051410,0.584906,5.598383,0.042227,2.157395,0.900528
2,(ROUND SNACK BOXES SET OF4 WOODLAND ),(PLASTERS IN TIN WOODLAND ANIMALS),0.185738,0.104478,0.056385,0.303571,2.905612,0.036979,1.285878,0.805439
3,(PLASTERS IN TIN WOODLAND ANIMALS),(ROUND SNACK BOXES SET OF4 WOODLAND ),0.104478,0.185738,0.056385,0.539683,2.905612,0.036979,1.768914,0.732353
4,(ROUND SNACK BOXES SET OF4 WOODLAND ),(ROUND SNACK BOXES SET OF 4 FRUITS ),0.185738,0.119403,0.099502,0.535714,4.486607,0.077325,1.896670,0.954379
5,(ROUND SNACK BOXES SET OF 4 FRUITS ),(ROUND SNACK BOXES SET OF4 WOODLAND ),0.119403,0.185738,0.099502,0.833333,4.486607,0.077325,4.885572,0.882486
6,(ROUND SNACK BOXES SET OF4 WOODLAND ),(SPACEBOY LUNCH BOX ),0.185738,0.077944,0.053068,0.285714,3.665653,0.038591,1.290879,0.893075
7,(SPACEBOY LUNCH BOX ),(ROUND SNACK BOXES SET OF4 WOODLAND ),0.077944,0.185738,0.053068,0.680851,3.665653,0.038591,2.551354,0.788669


In [433]:
print(my_basket_sets.columns)


Index([' 50'S CHRISTMAS GIFT BAG LARGE', ' DOLLY GIRL BEAKER',
       ' I LOVE LONDON MINI BACKPACK', ' RED SPOT GIFT BAG LARGE',
       ' SET 2 TEA TOWELS I LOVE LONDON ', ' SPACEBOY BABY GIFT SET',
       '10 COLOUR SPACEBOY PEN', '12 COLOURED PARTY BALLOONS',
       '12 IVORY ROSE PEG PLACE SETTINGS', '12 MESSAGE CARDS WITH ENVELOPES',
       ...
       'YULETIDE IMAGES GIFT WRAP SET', 'ZINC  HEART T-LIGHT HOLDER',
       'ZINC  STAR T-LIGHT HOLDER ', 'ZINC BOX SIGN HOME',
       'ZINC FOLKART SLEIGH BELLS', 'ZINC HEART LATTICE T-LIGHT HOLDER',
       'ZINC METAL HEART DECORATION', 'ZINC T-LIGHT HOLDER STAR LARGE',
       'ZINC T-LIGHT HOLDER STARS SMALL', 'ZINC WILLIE WINKIE  CANDLE STICK'],
      dtype='object', name='Description', length=1702)


In [435]:
my_basket_sets.columns = my_basket_sets.columns.str.strip()

In [437]:
# Filter rules with confidence >= 0.7 and lift >= 1.5
filtered_rules = my_rules[(my_rules['confidence'] >= 0.7) & (my_rules['lift'] >= 1.5)]

# View the filtered rules
print(filtered_rules)


                            antecedents  \
5  (ROUND SNACK BOXES SET OF 4 FRUITS )   

                             consequents  antecedent support  \
5  (ROUND SNACK BOXES SET OF4 WOODLAND )            0.119403   

   consequent support   support  confidence      lift  leverage  conviction  \
5            0.185738  0.099502    0.833333  4.486607  0.077325    4.885572   

   zhangs_metric  
5       0.882486  
